<a href="https://colab.research.google.com/github/Jaderfonseca/lime-smile-classifier/blob/main/neural_network_with_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Synthetic "smiley vs not_smile" dataset (clean mouth logic) ===
# Creates data/raw/{smile,not_smile}/*.png and data/raw/labels.csv

import os, csv, random, math
from pathlib import Path
import numpy as np
from PIL import Image, ImageDraw, ImageFilter

# ---- Config (tweak freely) ----
RANDOM_SEED    = 123
IMG_SIZE       = 48          # larger than 32 to reduce blockiness
N_PER_CLASS    = 500
OUT_DIR        = Path("data/raw")
CLASSES        = ["smile", "not_smile"]

STROKE         = 2           # line width for face/eyes/mouth
ADD_NOISE_P    = 0.12        # prob. of adding blur+noise (lower for cleaner icons)
NOISE_SIGMAS   = [0, 2, 3]   # std-dev options for Gaussian noise
BLUR_RADIUS    = 0.35

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

(OUT_DIR / "smile").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "not_smile").mkdir(parents=True, exist_ok=True)

def _clip(lo, hi, x):
    return max(lo, min(hi, x))

def add_noise(img, sigma):
    if sigma <= 0:
        return img
    arr   = np.array(img, dtype=np.int16)
    noise = np.random.normal(0, sigma, arr.shape)
    arr   = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(arr, mode="L")

def draw_mouth_curve(drw, x0, y0, x2, y2, cy, width, fg=0):
    """
    Quadratic Bezier from (x0,y0) to (x2,y2) with control ( (x0+x2)/2 , cy ).
    width = stroke width.
    """
    import numpy as np
    xs, ys = [], []
    cx = (x0 + x2) / 2.0
    for t in np.linspace(0, 1, 50):
        # Quadratic Bezier: B(t) = (1-t)^2 P0 + 2(1-t)t C + t^2 P2
        bx = (1-t)**2 * x0 + 2*(1-t)*t * cx + t**2 * x2
        by = (1-t)**2 * y0 + 2*(1-t)*t * cy + t**2 * y2
        xs.append(bx); ys.append(by)
    pts = list(map(tuple, np.stack([xs, ys], axis=1).astype(int)))
    drw.line(pts, fill=fg, width=width)

def draw_face(label="smile"):
    """Return a 48x48 grayscale face with a clear smile or not_smile."""
    W = H = IMG_SIZE
    bg, fg = 255, 0

    # Canvas
    img = Image.new("L", (W, H), color=bg)
    drw = ImageDraw.Draw(img)

    # Face (circle)
    r  = random.randint(int(W*0.35), int(W*0.44))
    cx = random.randint(int(W*0.48), int(W*0.52))
    cy = random.randint(int(H*0.48), int(H*0.52))
    bbox = [cx-r, cy-r, cx+r, cy+r]
    drw.ellipse(bbox, outline=fg, width=STROKE)

    # Eyes
    eye_r  = random.randint(3, 4)
    eye_dx = random.randint(6, 8)
    eye_y  = cy - random.randint(int(H*0.10), int(H*0.14))

    left_eye  = (cx - eye_dx - eye_r, eye_y - eye_r, cx - eye_dx + eye_r, eye_y + eye_r)
    right_eye = (cx + eye_dx - eye_r, eye_y - eye_r, cx + eye_dx + eye_r, eye_y + eye_r)
    drw.ellipse(left_eye,  fill=fg)
    drw.ellipse(right_eye, fill=fg)

    # Mouth baseline
    mouth_w = random.randint(int(W*0.50), int(W*0.66))
    mouth_y = random.randint(int(H*0.62), int(H*0.68))
    x0 = cx - mouth_w//2
    x2 = cx + mouth_w//2
    y0 = y2 = mouth_y

    # Curvature: positive -> control BELOW baseline (looks like a smile in image coords),
    # negative -> control ABOVE baseline (frown). 0 -> flat.
    base_curve = random.randint(6, 9)  # curvature magnitude in pixels (increase for more obvious)
    if label == "smile":
        cy_ctrl = mouth_y + base_curve + random.randint(0, 2)  # below baseline => U shape
        draw_mouth_curve(drw, x0, y0, x2, y2, cy_ctrl, width=STROKE, fg=fg)
    else:
        mode = random.choice(["frown", "flat"])
        if mode == "flat":
            drw.line([(x0, mouth_y), (x2, mouth_y)], fill=fg, width=STROKE)
        else:
            cy_ctrl = mouth_y - base_curve - random.randint(0, 2)  # above baseline => ∩
            draw_mouth_curve(drw, x0, y0, x2, y2, cy_ctrl, width=STROKE, fg=fg)

    # Mild blur + optional noise (kept small so shapes stay recognizable)
    if random.random() < ADD_NOISE_P:
        img = img.filter(ImageFilter.GaussianBlur(radius=BLUR_RADIUS))
        sigma = random.choice(NOISE_SIGMAS)
        img = add_noise(img, sigma=sigma)

    return img

# ---- Generate and save ----
rows = []
for label in CLASSES:
    for i in range(N_PER_CLASS):
        img   = draw_face(label=label)
        fname = f"{label}_{i:04d}.png"
        fpath = OUT_DIR / label / fname
        img.save(fpath)
        rows.append({"filepath": str(fpath.as_posix()), "label": label})

labels_csv = OUT_DIR / "labels.csv"
with open(labels_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=["filepath","label"])
    w.writeheader(); w.writerows(rows)

print(f"OK! Generated: {len(rows)} images")
print(f" - {sum(r['label']=='smile' for r in rows)} smiles")
print(f" - {sum(r['label']=='not_smile' for r in rows)} not_smile")
print(f"CSV: {labels_csv}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

df = pd.read_csv("data/raw/labels.csv")
sample = df.sample(25, random_state=42).reset_index(drop=True)

plt.figure(figsize=(6,6))
for i,(fp,lab) in enumerate(zip(sample["filepath"], sample["label"])):
    ax = plt.subplot(5,5,i+1)
    ax.imshow(Image.open(fp), cmap="gray")
    ax.set_title(lab, fontsize=8)
    ax.axis("off")
plt.tight_layout(); plt.show()


In [ ]:
# --- Preprocess: images -> flat numeric vectors ---

import json, csv, os
from pathlib import Path
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

RAW_DIR       = Path("data/raw")
PROC_DIR      = Path("data/processed")
LABELS_CSV    = RAW_DIR / "labels.csv"
IMG_SIZE      = 48          # must match generation (or detect from first img)
BINARIZE      = False       # set True to force 0/1 pixels
THRESHOLD     = 128         # used if BINARIZE=True
NORMALIZE     = True        # divide by 255 to [0,1]
RANDOM_SEED   = 42

PROC_DIR.mkdir(parents=True, exist_ok=True)

# 1) load table
rows = []
with open(LABELS_CSV, "r") as f:
    for r in csv.DictReader(f):
        rows.append(r)

print(f"Loaded {len(rows)} filepaths from {LABELS_CSV}")

# 2) encode labels
label_to_id = {"smile": 1, "not_smile": 0}
id_to_label = {v:k for k,v in label_to_id.items()}

# 3) loader + featurizer
def load_as_vector(path):
    img = Image.open(path).convert("L")
    if img.size != (IMG_SIZE, IMG_SIZE):
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.NEAREST)

    arr = np.array(img, dtype=np.uint8)

    if BINARIZE:
        arr = (arr >= THRESHOLD).astype(np.uint8) * 255

    if NORMALIZE:
        arr = arr.astype(np.float32) / 255.0
    else:
        arr = arr.astype(np.float32)

    return arr.flatten()  # shape: (IMG_SIZE*IMG_SIZE,)


In [ ]:
# build feature matrix
X_list, y_list = [], []

for r in rows:
    fp = r["filepath"]
    y_list.append(label_to_id[r["label"]])
    X_list.append(load_as_vector(fp))

X = np.vstack(X_list)               # (N, IMG_SIZE*IMG_SIZE)
y = np.array(y_list, dtype=np.int64)

print("Feature matrix:", X.shape, "labels:", y.shape)
print("Class balance: smiles=", (y==1).sum(), " | not_smile=", (y==0).sum())

# stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED
)

# save arrays
np.save(PROC_DIR / "X_train.npy", X_train)
np.save(PROC_DIR / "X_test.npy",  X_test)
np.save(PROC_DIR / "y_train.npy", y_train)
np.save(PROC_DIR / "y_test.npy",  y_test)

# save a tiny metadata companion for reproducibility
meta = {
    "img_size": IMG_SIZE,
    "binarize": BINARIZE,
    "threshold": THRESHOLD,
    "normalize": NORMALIZE,
    "random_seed": RANDOM_SEED,
    "n_total": int(X.shape[0]),
    "n_train": int(X_train.shape[0]),
    "n_test": int(X_test.shape[0]),
    "n_features": int(X.shape[1]),
    "label_to_id": label_to_id,
}
with open(PROC_DIR / "preprocess_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved to:", PROC_DIR)
for p in ["X_train.npy","X_test.npy","y_train.npy","y_test.npy","preprocess_meta.json"]:
    print(" -", PROC_DIR / p)


In [ ]:
# quick checks
Xt = np.load(PROC_DIR / "X_train.npy")
yt = np.load(PROC_DIR / "y_train.npy")

print("Shapes -> X_train:", Xt.shape, "| y_train:", yt.shape)
print("Means/std (first 5 cols):", Xt[:, :5].mean(axis=0), Xt[:, :5].std(axis=0))
print("Train balance:", (yt==1).sum(), "(smile) |", (yt==0).sum(), "(not_smile)")


In [ ]:
# --- Tiny MLP for smile vs not_smile (Keras/TF) ---

import os, json, random, numpy as np
import tensorflow as tf
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

PROC_DIR = Path("data/processed")
MODEL_DIR = Path("models"); MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR = Path("figures"); FIG_DIR.mkdir(parents=True, exist_ok=True)

# Load processed data
X_train = np.load(PROC_DIR / "X_train.npy")   # shape: (N, D)
X_test  = np.load(PROC_DIR / "X_test.npy")
y_train = np.load(PROC_DIR / "y_train.npy")   # shape: (N,)
y_test  = np.load(PROC_DIR / "y_test.npy")

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Scale inputs to [0,1] if not already
if X_train.max() > 1.0:
    X_train = X_train / 255.0
    X_test  = X_test  / 255.0

input_dim = X_train.shape[1]

# Build a small MLP (2 hidden layers)
from tensorflow.keras import layers, models, callbacks, optimizers

def make_model(input_dim: int) -> tf.keras.Model:
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(128, activation="relu")(inputs)
    x = layers.Dropout(0.15)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.10)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = make_model(input_dim)
model.summary()

# Callbacks: early stopping + best model checkpoint
ckpt_path = str(MODEL_DIR / "mlp_smile_best.keras")
cbs = [
    callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor="val_accuracy"),
    callbacks.ModelCheckpoint(ckpt_path, monitor="val_accuracy", save_best_only=True)
]

# Train
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=60,
    batch_size=64,
    callbacks=cbs,
    verbose=1
)

# Save final model and training history
model.save(MODEL_DIR / "mlp_smile_final.keras")
with open(MODEL_DIR / "mlp_smile_history.json", "w") as f:
    json.dump({k: [float(v) for v in vals] for k, vals in history.history.items()}, f)

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}  |  Test loss: {test_loss:.4f}")

# Predictions & reports
y_prob = model.predict(X_test, verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("\nClassification report:\n",
      classification_report(y_test, y_pred, target_names=["not_smile","smile"]))

cm = confusion_matrix(y_test, y_pred)

# Plot: training curves
plt.figure(figsize=(6,4))
plt.plot(history.history["accuracy"], label="train acc")
plt.plot(history.history["val_accuracy"], label="val acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("MLP accuracy")
plt.legend(); plt.tight_layout()
plt.savefig(FIG_DIR / "mlp_accuracy.png", dpi=160)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="val loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("MLP loss")
plt.legend(); plt.tight_layout()
plt.savefig(FIG_DIR / "mlp_loss.png", dpi=160)
plt.show()

# Plot: confusion matrix
plt.figure(figsize=(4.8,4))
plt.imshow(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.colorbar()
tick = np.arange(2)
plt.xticks(tick, ["not_smile","smile"])
plt.yticks(tick, ["not_smile","smile"])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha="center", va="center", fontsize=12)
plt.xlabel("Predicted"); plt.ylabel("True")
plt.tight_layout()
plt.savefig(FIG_DIR / "mlp_confusion_matrix.png", dpi=160)
plt.show()


In [ ]:
model.save("models/mlp_smile_classifier.keras")


In [ ]:
import os

os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

# salvar modelo e histórico
model.save("models/mlp_smile_classifier.keras")
np.save("results/history.npy", history.history)


In [ ]:
# --- 1) Compute metrics ---
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np, os

# predictions
proba = model.predict(X_test, verbose=0).ravel()
y_pred = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc:.3f}")

report = classification_report(y_test, y_pred, target_names=["not_smile","smile"])
print(report)

# save report to file
os.makedirs("results", exist_ok=True)
with open("results/classification_report.txt", "w") as f:
    f.write(f"Test accuracy: {acc:.3f}\n\n")
    f.write(report)

# --- 2) Confusion matrix ---
FIG_DIR = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(cm, cmap="Blues")
ax.figure.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(["not_smile","smile"]); ax.set_yticklabels(["not_smile","smile"])
ax.set_xlabel("Predicted"); ax.set_ylabel("True")
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha="center", va="center", fontsize=12)
ax.set_title("Confusion Matrix")
plt.tight_layout()
plt.savefig(f"{FIG_DIR}/confusion_matrix.png", dpi=160, bbox_inches="tight")
plt.show()

# --- 3) Accuracy per epoch (if history exists) ---
if os.path.exists("results/history.npy"):
    hist = np.load("results/history.npy", allow_pickle=True).item()
    train_acc = hist.get("accuracy", [])
    val_acc   = hist.get("val_accuracy", [])

    plt.figure(figsize=(6,4))
    if train_acc: plt.plot(train_acc, label="train acc")
    if val_acc:   plt.plot(val_acc,   label="val acc")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("MLP accuracy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{FIG_DIR}/accuracy.png", dpi=160, bbox_inches="tight")
    plt.show()


In [ ]:
# Minimal evaluation saver: writes files without displaying plots
import os, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

os.makedirs("results", exist_ok=True)
os.makedirs("figures", exist_ok=True)

# 1) Metrics → results/classification_report.txt
proba  = model.predict(X_test, verbose=0).ravel()
y_pred = (proba >= 0.5).astype(int)
acc    = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["not_smile","smile"])

with open("results/classification_report.txt", "w") as f:
    f.write(f"Test accuracy: {acc:.3f}\n\n")
    f.write(report)

# 2) Confusion matrix → figures/confusion_matrix.png
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(cm, cmap="Blues")
ax.figure.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(["not_smile","smile"]); ax.set_yticklabels(["not_smile","smile"])
ax.set_xlabel("Predicted"); ax.set_ylabel("True")
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha="center", va="center", fontsize=12)
ax.set_title("Confusion Matrix")
plt.tight_layout()
plt.savefig("figures/confusion_matrix.png", dpi=160, bbox_inches="tight")
plt.close(fig)

# 3) Accuracy per epoch (if history exists) → figures/accuracy.png
if os.path.exists("results/history.npy"):
    hist = np.load("results/history.npy", allow_pickle=True).item()
    train_acc = hist.get("accuracy", [])
    val_acc   = hist.get("val_accuracy", [])

    fig2, ax2 = plt.subplots(figsize=(6,4))
    if train_acc: ax2.plot(train_acc, label="train acc")
    if val_acc:   ax2.plot(val_acc,   label="val acc")
    ax2.set_xlabel("Epoch"); ax2.set_ylabel("Accuracy"); ax2.set_title("MLP accuracy")
    ax2.legend()
    plt.tight_layout()
    plt.savefig("figures/accuracy.png", dpi=160, bbox_inches="tight")
    plt.close(fig2)


In [ ]:
# --- First LIME explanation (48x48 grayscale) ---

# 1) install & imports
!pip -q install lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
from keras.models import load_model
import numpy as np, matplotlib.pyplot as plt, os

# 2) config & load artifacts
IMG_SIZE = 48                        # your images are 48x48
MODEL_PATH = "models/mlp_smile_final.keras"
X_TEST = "data/processed/X_test.npy"
Y_TEST = "data/processed/y_test.npy"
FIG_DIR = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

model = load_model(MODEL_PATH)
X_test = np.load(X_TEST)            # shape: (N, 48*48)
y_test = np.load(Y_TEST)            # 0 = not_smile, 1 = smile

# LIME sends RGB (48x48x3). Convert to grayscale and flatten. ---
import numpy as np

def predict_fn(images):
  arr = np.array(images)

  # if LIME gives RGB, convert to grayscale by channel mean
  if arr.ndim == 4 and arr.shape[-1] == 3: #(B, 48, 48, 3)
      arr = arr.mean(axis=-1)             # ->(B, 48, 48)

  # Ensure shape is (B, 48, 48)
  if arr.ndim == 2:                       # single image (48, 48)
      arr = arr[None, ...]

  # Scale to [0,1] if values likely in 0..255
  if arr.max() > 1.5:
      arr = arr / 255.0

  flat = arr.reshape(len(arr), -1)                  # (B, 2304)
  p_smile = model.predict(flat, verbose=0).reshape(-1, 1)  # sigmoid output
  return np.hstack([1 - p_smile, p_smile])

# 4) pick one positive and one negative example to test
pos_idx = int(np.where(y_test == 1)[0][0])    # first "smile"
neg_idx = int(np.where(y_test == 0)[0][0])    # first "not_smile"

def run_lime(idx: int, tag: str):
    """Generate one LIME explanation and save to figures/lime_{tag}.png"""
    sample = X_test[idx].reshape(IMG_SIZE, IMG_SIZE)

    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(
        image=sample,                 # 2D grayscale
        classifier_fn=predict_fn,     # returns (B, 2) probs
        labels=[0, 1],                # 0=not_smile, 1=smile
        num_samples=800,              # increase for more stability (slower)
    )

    # visualize contributions for the true label
    temp, mask = explanation.get_image_and_mask(
        label=int(y_test[idx]),
        positive_only=False,          # show both positive/negative regions
        hide_rest=False,
        num_features=6,               # number of superpixels to show
        min_weight=0.0,
    )

    plt.figure(figsize=(4,4))
    plt.imshow(mark_boundaries(temp, mask), cmap="gray")
    plt.title(f"LIME — idx {idx} | true: {'smile' if y_test[idx]==1 else 'not_smile'}")
    plt.axis("off")
    out = os.path.join(FIG_DIR, f"lime_{tag}.png")
    plt.savefig(out, dpi=180, bbox_inches="tight")
    plt.show()
    print(f"Saved: {out}")

# 5) run once for each class
run_lime(pos_idx, "sample_smile")
run_lime(neg_idx, "sample_not_smile")


In [ ]:
import os
import numpy as np
import json

# Make sure the folders exist
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

# --- Save model ---
model.save("models/mlp_smile_final.keras")

# --- Save training history (as .npy) ---
np.save("results/history.npy", history.history)

# --- Save training history (as .json) ---
with open("results/mlp_history.json", "w") as f:
    json.dump(history.history, f)


In [ ]:
# Batch LIME Explanations (save as exp_sample#.png)

!pip -q install lime
from lime import lime_image
from skimage.segmentation import slic, mark_boundaries
from keras.models import load_model
import numpy as np, matplotlib.pyplot as plt, os

# --- configuration ---
IMG_SIZE     = 48
MODEL_PATH   = "models/mlp_smile_final.keras"
X_TEST_PATH  = "data/processed/X_test.npy"
Y_TEST_PATH  = "data/processed/y_test.npy"
FIG_DIR      = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

N_TOTAL      = 10        # total explanations to generate
NUM_SAMPLES  = 800       # higher = more stable, but slower
NUM_FEATURES = 6         # number of superpixels highlighted
RANDOM_STATE = 42

# --- load model and data ---
model  = load_model(MODEL_PATH)
X_test = np.load(X_TEST_PATH, allow_pickle=True)
y_test = np.load(Y_TEST_PATH, allow_pickle=True).astype(int)

def to_2d(xrow):
    return xrow.reshape(IMG_SIZE, IMG_SIZE) if xrow.ndim == 1 else xrow

# Balanced sampling across classes
rng = np.random.default_rng(RANDOM_STATE)
idx_pos = rng.permutation(np.where(y_test == 1)[0]).tolist()
idx_neg = rng.permutation(np.where(y_test == 0)[0]).tolist()
half = N_TOTAL // 2
picked = idx_pos[:half] + idx_neg[:(N_TOTAL - half)]
rng.shuffle(picked)

# Predictor for LIME
def predict_fn(images):
    arr = np.array(images)
    if arr.ndim == 3:                 # single image
        arr = arr[None, ...]
    arr = arr.astype("float32")
    if arr.max() > 1.5:
        arr = arr / 255.0
    if arr.shape[-1] == 3:            # RGB -> grayscale
        arr = arr.mean(axis=-1)
    flat = arr.reshape(len(arr), -1)
    p_smile = model.predict(flat, verbose=0).reshape(-1, 1)  # sigmoid
    return np.hstack([1 - p_smile, p_smile])                 # [not_smile, smile]

# Init explainer with seed (do NOT pass random_state to explain_instance)
explainer = lime_image.LimeImageExplainer(random_state=RANDOM_STATE)

def explain_and_save(idx, out_path):
    img2d = to_2d(X_test[idx])

    # Robust SLIC (works with new and old scikit-image)
    def seg_fn(x):
        x = np.asarray(x)
        if x.ndim == 2:
            x = np.repeat(x[..., None], 3, axis=-1)
        x = x.astype("float32")
        if x.max() > 1.5:
            x = x / 255.0
        try:
            return slic(x, n_segments=50, compactness=10, start_label=0, channel_axis=-1)
        except TypeError:
            # older scikit-image versions
            return slic(x, n_segments=50, compactness=10, start_label=0)

    exp = explainer.explain_instance(
        image=img2d,                   # 2D grayscale is fine
        classifier_fn=predict_fn,
        labels=[0, 1],
        num_samples=NUM_SAMPLES,
        segmentation_fn=seg_fn
    )

    # Model prediction for title
    rgb = np.repeat(img2d[..., None], 3, axis=-1)
    probs = predict_fn(rgb[None, ...])[0]
    pred = int(np.argmax(probs))
    p_smile = float(probs[1])

    # Visualize explanation for the TRUE label
    true_label = int(y_test[idx])
    temp, mask = exp.get_image_and_mask(
        label=true_label,
        positive_only=False,
        hide_rest=False,
        num_features=NUM_FEATURES,
        min_weight=0.0,
    )

    plt.figure(figsize=(4,4))
    plt.imshow(mark_boundaries(temp, mask), cmap="gray")
    title = f"idx {idx} | true: {'smile' if true_label==1 else 'not_smile'} | pred: {'smile' if pred==1 else 'not_smile'} ({p_smile:.2f})"
    plt.title(f"LIME — {title}")
    plt.axis("off")
    plt.savefig(out_path, dpi=180, bbox_inches="tight")
    plt.close()

# Run
for i, idx in enumerate(picked, start=1):
    out_file = os.path.join(FIG_DIR, f"exp_sample{i}.png")
    explain_and_save(idx, out_file)
    print(f"Saved: {out_file}")


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os, glob
import numpy as np

FIG_DIR = "figures"

# Collect all generated explanation PNGs
files = sorted(glob.glob(os.path.join(FIG_DIR, "exp_sample*.png")))

# Map each file back to its index (idx was included in the plot title, but filenames are sequential)
# We'll rely on the order from `picked` if you kept that variable
# If not, you can just separate manually later
print(f"Found {len(files)} files")

def make_grid(image_paths, out_path, title, cols=4):
    rows = int(np.ceil(len(image_paths) / cols))
    plt.figure(figsize=(cols*3, rows*3))
    for i, f in enumerate(image_paths, 1):
        img = imread(f)
        plt.subplot(rows, cols, i)
        plt.imshow(img)
        plt.axis("off")
        plt.title(os.path.basename(f), fontsize=8)
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close()
    print(f"Saved grid: {out_path}")

# Separate by class using y_test and picked
smile_files = []
not_smile_files = []

for i, idx in enumerate(picked, start=1):
    f = os.path.join(FIG_DIR, f"exp_sample{i}.png")
    if not os.path.exists(f):
        continue
    if y_test[idx] == 1:
        smile_files.append(f)
    else:
        not_smile_files.append(f)

# Generate grids
make_grid(smile_files, os.path.join(FIG_DIR, "exp_grid_smile.png"), "LIME Explanations — Smile")
make_grid(not_smile_files, os.path.join(FIG_DIR, "exp_grid_not_smile.png"), "LIME Explanations — Not Smile")


In [ ]:
import numpy as np

# Get model predictions on the full test set
flat_X = X_test.reshape(len(X_test), -1) if X_test.ndim > 2 else X_test
probs = model.predict(flat_X, verbose=0).reshape(-1)
preds = (probs >= 0.5).astype(int)

# Identify correct and incorrect indices
correct_idx = np.where(preds == y_test)[0]
incorrect_idx = np.where(preds != y_test)[0]

print(f"Correct predictions: {len(correct_idx)}")
print(f"Incorrect predictions: {len(incorrect_idx)}")

# Example: pick a few incorrect cases for inspection
sample_errors = incorrect_idx[:5]

print("Sample misclassified indices:", sample_errors)


In [ ]:
for j, idx in enumerate(sample_errors, start=1):
    out_path = os.path.join(FIG_DIR, f"exp_error{j}.png")
    explain_and_save(idx, out_path)
    print(f"Saved error explanation: {out_path}")


In [17]:
# Remove folder sample_data created by Colab
import shutil
import os

if os.path.exists("sample_data"):
    shutil.rmtree("sample_data")
    print("sample_data folder removed successfully.")
else:
    print("No sample_data folder found.")


sample_data folder removed successfully.


In [18]:
# === Minimal repo materializer (lime-smile-classifier) ===
import os, glob, shutil, numpy as np

BASE = "."
DIRS = {
    "raw":        "data/raw",
    "proc":       "data/processed",
    "fig":        "figures",
    "res":        "results",
    "models":     "models",
    "nb":         "notebooks",
}
for d in DIRS.values(): os.makedirs(d, exist_ok=True)
open(os.path.join(DIRS["raw"], ".gitkeep"), "a").close()  # keep raw/ in Git

# 1) Save arrays if present
def save_npy(name, arr):
    dst = os.path.join(DIRS["proc"], name); np.save(dst, arr); print("Saved:", dst)

for var, fname in [("X_train","X_train.npy"),("y_train","y_train.npy"),
                   ("X_test","X_test.npy"),("y_test","y_test.npy")]:
    if var in globals(): save_npy(fname, globals()[var])

# 2) Move & normalize figures
RENAME = {
    "mlp_accuracy.png":         "training_accuracy.png",
    "accuracy.png":             "training_accuracy.png",
    "mlp_loss.png":             "training_loss.png",
    "mlp_confusion_matrix.png": "confusion_matrix.png",
}
def move_figure(path):
    name = os.path.basename(path)
    name = RENAME.get(name, name)  # normalize common variants
    dst  = os.path.join(DIRS["fig"], name)
    if os.path.abspath(path) != os.path.abspath(dst):
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(path, dst); print("Moved →", dst)

for pat in ("*.png", "notebooks/*.png"):
    for p in glob.glob(pat):
        if os.path.commonpath([os.path.abspath(p), os.path.abspath(DIRS["fig"])]) == os.path.abspath(DIRS["fig"]):
            continue
        move_figure(p)

# 3) Move models
for p in glob.glob("*.keras")+glob.glob("*.h5"):
    dst = os.path.join(DIRS["models"], os.path.basename(p))
    if os.path.abspath(p)!=os.path.abspath(dst): shutil.move(p,dst); print("Moved →", dst)

# 4) Move results (keep README/requirements at root)
SKIP = {"readme.md","requirements.txt"}
for pat in ("*.txt","*.npy","*.json"):
    for p in glob.glob(pat):
        if os.path.basename(p).lower() in SKIP: continue
        dst = os.path.join(DIRS["res"], os.path.basename(p))
        if os.path.abspath(p)!=os.path.abspath(dst): shutil.move(p,dst); print("Moved →", dst)

# 5) Move notebooks from root
for nb in glob.glob("*.ipynb"):
    dst = os.path.join(DIRS["nb"], os.path.basename(nb))
    if os.path.abspath(nb)!=os.path.abspath(dst): shutil.move(nb,dst); print("Moved →", dst)

# 6) Print tree
def tree(root="."):
    for dp, dn, fn in os.walk(root):
        dn[:]  = [d for d in dn if not d.startswith(".")]
        fn[:]  = [f for f in fn if not f.startswith(".")]
        ind = "    " * (dp.replace(root, "").count(os.sep))
        print(f"{ind}{os.path.basename(dp)}/")
        for f in sorted(fn): print(f"{ind}    {f}")

print("\n=== Repository tree ===")
tree(BASE)


Saved: data/processed/X_train.npy
Saved: data/processed/y_train.npy
Saved: data/processed/X_test.npy
Saved: data/processed/y_test.npy

=== Repository tree ===
./
    figures/
        accuracy.png
        confusion_matrix.png
        exp_error1.png
        exp_grid_not_smile.png
        exp_grid_smile.png
        exp_sample1.png
        exp_sample10.png
        exp_sample2.png
        exp_sample3.png
        exp_sample4.png
        exp_sample5.png
        exp_sample6.png
        exp_sample7.png
        exp_sample8.png
        exp_sample9.png
        lime_sample_not_smile.png
        lime_sample_smile.png
        mlp_accuracy.png
        mlp_confusion_matrix.png
        mlp_loss.png
    models/
        mlp_smile_best.keras
        mlp_smile_classifier.keras
        mlp_smile_final.keras
        mlp_smile_history.json
    data/
        raw/
            labels.csv
            not_smile/
                not_smile_0000.png
                not_smile_0001.png
                not_smile_0002.png